In [70]:
import requests as re
import pandas as pd
import datetime
import altair as alt
import altair_latimes as lat

In [71]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Download
Source: [California Water Watch](https://cww.water.ca.gov/), PRISM

Some parameters

In [72]:
today = datetime.date.today()
today

datetime.date(2024, 2, 1)

In [73]:
one_year_ago = datetime.datetime.today() - datetime.timedelta(days=365)

In [74]:
atlasID="Statewide"
varKey="PPT_YTD"

Query

In [75]:
url = f'https://cww.water.ca.gov/service/prism/huc8/precipstats?date={today}&atlasID={atlasID}&varKey={varKey}'

In [76]:
data = re.get(url).json()['data']

Query one year ago

In [77]:
last_year_url = f'https://cww.water.ca.gov/service/prism/huc8/precipstats?date={one_year_ago}&atlasID={atlasID}&varKey={varKey}'

In [78]:
last_year_data = re.get(last_year_url).json()['data']

### Parse

In [79]:
def parsePrecipStats(data):
    columns = data['seriesNames']
    columns.insert(0, "date")
    
    df = pd.DataFrame(data['series'], columns=columns)
    df.columns = df.columns.str.lower()
    df['date'] = pd.to_datetime(df['date'])
    
    return df

In [80]:
df = parsePrecipStats(data)

In [81]:
last_year_df = parsePrecipStats(last_year_data)

## Get toplines

In [82]:
prcp_min=data['min']
prcp_min

2.31

In [83]:
prcp_avg=data['avg']
prcp_avg

12.36

In [84]:
prcp_max=data['max']
prcp_max

23.54

In [85]:
year_end_avg = data['yearEndAvg']
year_end_avg

23.68

### Plot

In [86]:
gauge = df[
    (~df.current.isna())
]

In [87]:
gauge = gauge[gauge.date == gauge.date.max()]

In [88]:
gauge["record"] = prcp_max

In [89]:
gauge["pct_of_avg"] = (gauge["current"] / gauge["average"]) * 100

Last year

In [90]:
total_last_year = last_year_df[ last_year_df.date == gauge.date.max() ].iloc[0]["current"]

In [91]:
gauge["one_year_ago"] = total_last_year

In [93]:
alt.Chart(
    pd.melt(gauge, id_vars="date", value_vars=["current","average","one_year_ago"])
).mark_bar().encode(
    x="value",
    y=alt.Y("variable", sort="-x"),
    color="variable"
).properties(
    title="Year-to-date precipitation in California compared to average",
    width=600,
    height=200
)

alt.Chart(...)

In [ ]:
line_chart = pd.melt(df, id_vars="date", value_vars=["current","average"])

In [ ]:
alt.Chart(line_chart).mark_line().encode(
    x="date:T",
    y="value",
    color="variable",
).properties(
    title=f"California precipitation in {today.year} compared to average",
    width=600
)

### Export

In [ ]:
gauge.to_csv("../../data/processed/precipitation/latest.csv", index=False)